# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117321e+02     1.448079e+00
 * time: 0.6722080707550049
     1     1.052856e+01     9.192584e-01
 * time: 1.7243189811706543
     2    -1.179296e+01     9.656650e-01
 * time: 2.2277960777282715
     3    -3.410818e+01     7.548330e-01
 * time: 2.9831528663635254
     4    -4.757392e+01     5.445843e-01
 * time: 3.715108871459961
     5    -5.700694e+01     2.064073e-01
 * time: 4.449311017990112
     6    -5.984354e+01     1.214770e-01
 * time: 4.9591429233551025
     7    -6.089312e+01     5.082017e-02
 * time: 5.455082893371582
     8    -6.130376e+01     6.274855e-02
 * time: 5.950934886932373
     9    -6.159409e+01     3.586553e-02
 * time: 6.446707010269165
    10    -6.180805e+01     2.521219e-02
 * time: 6.951552867889404
    11    -6.196599e+01     2.073727e-02
 * time: 7.4475390911102295
    12    -6.203069e+01     1.718458e-02
 * time: 7.944231986999512
    13    -6.209324e+01     1.819502e-02
 * time: 8.439349889755249
 

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671095
    AtomicLocal         -18.8557767
    AtomicNonlocal      14.8522683
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485411 
    Xc                  -19.3336836

    total               -62.261666460542